In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/root/mmdet3d


In [3]:
from mmdet3d.datasets import build_dataset, build_dataloader
from mmdet3d.models import build_model
from mmdet3d.utils import recursive_eval
from mmcv import Config
from torchpack.utils.config import configs
import random
import numpy as np
import torch
from mmdet.datasets import DATASETS
from mmdet3d.datasets.dataset_wrappers import CBGSDataset
import os
from torchvision.utils import save_image
from mmdet3d.core.utils import visualize_lidar
from mmdet3d.core.utils.visualize import visualize_lidar_combined
import copy

In [4]:
def find_in_pipeline(pipeline: list, type: str) -> int:
    for i, p in enumerate(pipeline):
        if p["type"] == type:
            return i
    return -1


def run_temporal_loading(cfg, dataloader, stop_batch_idx=None, save_path=None):
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)
        torch.manual_seed(cfg.seed)

    for batch_idx, batch_data in enumerate(dataloader):
        frame_idxs = [x["frame_idx"] for x in batch_data["metas"].data[0][0]]
        print(f"batch_idx: {batch_idx:<4} seq_indices:", frame_idxs)
        
        if save_path is not None:
            save_visuals(cfg, batch_idx, batch_data, save_path)

        if stop_batch_idx is not None and batch_idx == stop_batch_idx:
            return


def save_visuals(cfg, batch_idx:int, batch_data: dict, target_path: str) -> None:
    batch_size = batch_data["img"].data[0].size(0)
    for batch_i_idx in range(batch_size):
        img = batch_data["img"].data[0][batch_i_idx]
        points = batch_data["points"].data[0][batch_i_idx]
        metas = batch_data["metas"].data[0][batch_i_idx]
        gt_bboxes_3d = batch_data["gt_bboxes_3d"].data[0][batch_i_idx]
        gt_labels_3d = batch_data["gt_labels_3d"].data[0][batch_i_idx]

        # Temporal Frame, Image, C, H, W
        assert len(img.shape) == 5, img.shape

        len_queue = img.size(0)
        len_img = img.size(1)
        sampled_bboxes_3d = copy.deepcopy(gt_bboxes_3d)
        save_root_id = f"b{str(batch_idx).zfill(4)}_{str(batch_i_idx).zfill(2)}"
        seq_indices = [str(x["frame_idx"]).zfill(3) for x in metas]

        lidar_root_combined_path = os.path.join(target_path, "lidar-combined")
        lidar_root_classes_path = os.path.join(target_path, "lidar-classes")
        img_root_paths = [os.path.join(target_path, f"images-{img_id}") for img_id in range(len_img)]
        [os.makedirs(x, exist_ok=True) for x in img_root_paths]

        all_bboxes_3d = copy.deepcopy(gt_bboxes_3d)
        all_labels_3d = copy.deepcopy(gt_labels_3d)
        for i in range(len_queue):
            sample_mode:list = metas[i]["sample_mode"] # list of 0 and 1
            # switch 0 and 1s
            mask = np.array(~sample_mode + 2, dtype=np.bool_)
            gt_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[mask])
            sampled_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[~mask])
            gt_labels_3d[i] = all_labels_3d[i][mask]


        for i in range(len_queue):
            save_id = f"{save_root_id}_{seq_indices[i]}.jpg"
            for img_id in range(img.shape[1]):
                img_ = img[i][img_id]
                save_image(img_, os.path.join(img_root_paths[img_id], save_id))

            visualize_lidar(
                os.path.join(lidar_root_classes_path, save_id),
                points[i],
                bboxes=all_bboxes_3d[i],
                labels=all_labels_3d[i],
                classes=cfg.object_classes,
                dataset=cfg.data.train.dataset.type,
                xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
                ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
            )
            
            visualize_lidar_combined(
                os.path.join(lidar_root_combined_path, save_id),
                points[i],
                pred_bboxes=sampled_bboxes_3d[i],
                gt_bboxes=gt_bboxes_3d[i],
                xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
                ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
            )

In [5]:
config_path = (
    "configs/tumtraf-i/temporal/transfusion/lidar/voxelnet-1600g-0xy1-0z20-ql3-qrt2-gtp15.yaml"
)
configs.load(config_path, recursive=True)
cfg = Config(recursive_eval(configs), filename=config_path)

## Train Pipeline


In [8]:
copy_cfg = copy.deepcopy(cfg)

# disable ImageNormalize
# in_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ImageNormalize")
# copy_cfg.data.train.dataset.pipeline[in_idx]["skip_normalize"] = True

# # find RandomFlip3D
# rf_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "RandomFlip3D")
# copy_cfg.data.train.dataset.pipeline[rf_idx]["flip_horizontal"] = False

# # find GlobalRotScaleTrans
# grst_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "GlobalRotScaleTrans")
# copy_cfg.data.train.dataset.pipeline[grst_idx]["is_train"] = False

# gtp_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ObjectPaste")
# copy_cfg.data.train.dataset.pipeline[gtp_idx]["apply_collision_check"] = True
# copy_cfg.data.train.dataset.pipeline[gtp_idx]["apply_same_aug_to_seq"] = True
# copy_cfg.data.train.dataset.pipeline[gtp_idx]["apply_temporal_forward"] = False
# copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.cls_trans_lim = {
#     "CAR": ["uniform", 0.4, 2.0],
#     "TRAILER": ["uniform", 0.4, 1.5],
#     "TRUCK": ["uniform", 0.4, 1.5],
#     "VAN": ["uniform", 0.4, 0.5],
#     "PEDESTRIAN": ["uniform", 0.4, 1.0],
#     "BUS": ["uniform", 0.0, 0.5],
#     "MOTORCYCLE": ["uniform", 0.4, 1.5],
#     "BICYCLE": ["uniform", 0.4, 1.2],
#     "EMERGENCY_VEHICLE": ["uniform", 0.4, 0.15],
# }
# copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.cls_rot_lim = {
#     "CAR": ["normal", 0.0, 0.15],
#     "TRAILER": ["normal", 0.0, 0.1],
#     "TRUCK": ["normal", 0.0, 0.1],
#     "VAN": ["normal", 0.0, 0.1],
#     "PEDESTRIAN": ["normal", 0.0, 0.3],
#     "BUS": ["normal", 0.0, 0.1],
#     "MOTORCYCLE": ["normal", 0.0, 0.2],
#     "BICYCLE": ["normal", 0.0, 0.2],
#     "EMERGENCY_VEHICLE": ["normal", 0.0, 0.1],
# }


dataset = build_dataset(copy_cfg.data.train, dict(test_mode=False))
dataloader = build_dataloader(dataset, 1, 1, 1, dist=True, seed=copy_cfg.seed, shuffle=True)

save_path = os.path.join("debugging-results-delete", "tumtraf-i-temporal-loading")
os.system(f"rm -rf {save_path}")
run_temporal_loading(cfg, dataloader, stop_batch_idx=5, save_path=save_path)

num_gt: 22
num_sampled: 15
valid samples: 15 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
not valid samples: 0 []
num_gt: 21
num_sampled: 15
valid samples: 13 [0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14]
not valid samples: 2 [2, 6]
reverting translation
reverting rotation
num_gt: 20
num_sampled: 15
valid samples: 13 [0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14]
not valid samples: 2 [2, 6]
reverting translation
reverting rotation
num_gt: 16
num_sampled: 15
valid samples: 15 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
not valid samples: 0 []
num_gt: 16
num_sampled: 15
valid samples: 13 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]
not valid samples: 2 [1, 11]
reverting translationreverting rotation

num_gt: 16num_sampled: 15


valid samples: 13 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]not valid samples: 2 [1, 11]
reverting translation
reverting rotation
batch_idx: 0    seq_indices: [50, 51, 52]
core dict_keys(['img', 'points', 'gt_bboxes_3d', 'gt_labels_3d', 'camera_intrinsic

## Test Pipeline


In [7]:
# dataset = build_dataset(cfg.data.test, dict(test_mode=True))
# dataloader = build_dataloader(dataset, 1, 4, 1, dist=True, seed=cfg.seed, shuffle=False)
# run_temporal_loading(cfg, dataloader, 1)